In [1]:
import pandas as pd

In [2]:
lookups = pd.read_csv('data/CRMLS_mapping_fields_and_lookups.csv', delimiter='\t')
lookups.head()

,resource,field_long,field_short,searchable,data_type,lookup_long,lookup_short,raw_value
0,Property,City,City,True,Character,Wrightwood,WRIW,WRIW
1,Property,City,City,True,Character,Acton,AC,AC
2,Property,City,City,True,Character,Agoura,AGO,AGO
3,Property,City,City,True,Character,Agoura Hills,AGHI,AGHI
4,Property,City,City,True,Character,Agua Dulce,ADUL,ADUL


In [3]:
lookups[lookups['field_long'] == 'City']

,resource,field_long,field_short,searchable,data_type,lookup_long,lookup_short,raw_value
0,Property,City,City,True,Character,Wrightwood,WRIW,WRIW
1,Property,City,City,True,Character,Acton,AC,AC
2,Property,City,City,True,Character,Agoura,AGO,AGO
3,Property,City,City,True,Character,Agoura Hills,AGHI,AGHI
4,Property,City,City,True,Character,Agua Dulce,ADUL,ADUL
...,...,...,...,...,...,...,...,...
1581,Property,City,City,True,Character,Robinsons Corner,ROBC,ROBC
1582,Property,City,City,True,Character,Frenchtown,FRNC,FRNC
1583,Property,City,City,True,Character,Strawberry Valley,SWVA,SWVA
1584,Property,City,City,True,Character,Bear Reservoir,BRES,BRES


In [4]:
def geocode(data, api_key):
    """

    Args:
        data (_type_): _description_
        api_key (_type_): _description_

    Returns:
        _type_: _description_
    """
    from datetime import datetime
    import pandas as pd
    import requests

    print(f"Started geocoding {data['parsed_add'].nunique()} cities")
        # Required fields in data df: 'l_id'; 'parsed_add'
    
    now = datetime.now()
    log_time = str(now.strftime("%m_%d_%Y"))
    n=0
    retries =[]
    for i in range(len(data)):
        # Set the address that you want to geocode
        address = data['parsed_add'][i]
            
            # Build the API request URL with the address and boundary
        url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"

            # Send the request and get the response
        response = requests.get(url).json()

        data.loc[i, 'status'] = response["status"]
        try:
                # Get the latitude and longitude from the response
            location = response['results'][0]['geometry']['location']
            latitude = location['lat']
            longitude = location['lng']
            data.loc[i, 'lat'] = latitude
            data.loc[i, 'long'] = longitude 
        except Exception as e:
            retries.append(data['parsed_add'][i])

        n += 1
        if n % 100 == 0:  # Check if current index is a multiple of 500
            print(f"Processed {n} cities")
            #print(data.loc[i])
            data.to_csv(f'data/result_geocode_{log_time}.csv',index=False)
        
    # Iterate through the list and write each element to the file
    with open(f'data/failed_ids_{log_time}.txt', 'w') as file:
        for item in retries:
            file.write("%s\n" % item)
    data.to_csv(f'data/result_geocode_{log_time}.csv',index=False)
    return data, retries

In [5]:
d = pd.DataFrame({'parsed_add': [x + ' California' for x in lookups[lookups['field_long'] == 'City']['lookup_long']],
                  'lat': [0] * len(lookups[lookups['field_long'] == 'City']),
                  'long': [0] * len(lookups[lookups['field_long'] == 'City']),
                  'status': [None] * len(lookups[lookups['field_long'] == 'City'])})
import json
with open('data/api_keys/keys.json', 'r') as f:
    k = json.load(f)
d, retries = geocode(d, k['google_maps'])

Started geocoding 1586 cities


/var/folders/6l/yzmcq93j41zf3wrth_tyl6g80000gn/T/ipykernel_12872/2739824363.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '34.3608341' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, 'lat'] = latitude
/var/folders/6l/yzmcq93j41zf3wrth_tyl6g80000gn/T/ipykernel_12872/2739824363.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-117.6333884' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, 'long'] = longitude


Processed 100 cities
parsed_add    Pacific Palisades California
lat                              34.046737
long                           -118.546434
status                                  OK
Name: 99, dtype: object
Processed 200 cities
parsed_add    Bell Gardens California
lat                         33.965292
long                      -118.151459
status                             OK
Name: 199, dtype: object
Processed 300 cities
parsed_add    Los Angeles California
lat                        34.054908
long                     -118.242643
status                            OK
Name: 299, dtype: object
Processed 400 cities
parsed_add    Romona California
lat                   33.037638
long                -116.876539
status                       OK
Name: 399, dtype: object
Processed 500 cities
parsed_add    Buellton California
lat                     34.613597
long                   -120.19265
status                         OK
Name: 499, dtype: object
Processed 600 cities
parsed_add    

In [8]:
d.head()

,parsed_add,lat,long,status
0,Wrightwood California,34.360834,-117.633388,OK
1,Acton California,34.469994,-118.196742,OK
2,Agoura California,34.153340,-118.761676,OK
3,Agoura Hills California,34.153340,-118.761676,OK
4,Agua Dulce California,34.496344,-118.326197,OK


In [9]:
d['status'].unique()

array(['OK', 'ZERO_RESULTS'], dtype=object)

In [13]:
import geopandas as gpd
import shapely
from tqdm.auto import tqdm
crmls = pd.read_csv('data/all_listings_lat_long_city_crmls.csv')
for i in range(2,47):
    crmls = pd.concat([crmls, pd.read_csv(f'data/all_listings_lat_long_city_crmls_{i}.csv')])
print('done loading data')
crmls["geometry"] = [
    shapely.Point(long, lat)
    for long, lat in tqdm(zip(crmls["longitude"], crmls["latitude"]), total=len(crmls))
]
crmls = gpd.GeoDataFrame(crmls, crs="EPSG:4326")

done loading data


  0%|          | 0/11178605 [00:00<?, ?it/s]

In [14]:
crmls.head()

,mls_area_major,city,county_or_parish,latitude,longitude,geometry
0,61,STAN,OR,33.784077,-117.989219,POINT (-117.98922 33.78408)
1,82,BP,OR,33.849421,-117.983828,POINT (-117.98383 33.84942)
2,CS,CDM,OR,33.591342,-117.857918,POINT (-117.85792 33.59134)
3,TUL,TUL,LA,34.144482,-118.364059,POINT (-118.36406 34.14448)
4,M3,LAM,LA,33.907450,-117.992478,POINT (-117.99248 33.90745)


In [20]:
d['centroid'] = [
    shapely.Point(long, lat)
    for long, lat in tqdm(zip(d["long"], d["lat"]), total=len(d))
]

  0%|          | 0/1586 [00:00<?, ?it/s]

In [22]:
d.columns = ['City', 'latitude', 'longitude', 'api_status', 'centroid']
d['City'] = [x[:-11] for x in d['City']]
d.head()

,City,latitude,longitude,api_status,centroid
0,Wrightwood,34.360834,-117.633388,OK,POINT (-117.6333884 34.3608341)
1,Acton,34.469994,-118.196742,OK,POINT (-118.1967418 34.469994)
2,Agoura,34.153340,-118.761676,OK,POINT (-118.7616764 34.1533395)
3,Agoura Hills,34.153340,-118.761676,OK,POINT (-118.7616764 34.1533395)
4,Agua Dulce,34.496344,-118.326197,OK,POINT (-118.3261969 34.4963443)


In [28]:
crmls = crmls.merge(lookups[['lookup_long', 'lookup_short']], left_on='city', right_on='lookup_short')

In [98]:
def get_radius(
    listings: gpd.GeoDataFrame, centroid: shapely.Point, thresh=0.75, radius_incr=0.005
) -> float:
    if len(listings) == 0:
        return 0
    if centroid == shapely.Point(0, 0) or pd.isna(centroid):
        centroid = shapely.MultiPoint(listings["geometry"].values).centroid
    
    radius = 0 + radius_incr
    check = gpd.GeoDataFrame(
        {"check": [0], "geometry": [centroid.buffer(radius)]}, crs="EPSG:4326"
    )
    while len(listings.sjoin(check, predicate="intersects")) / len(listings) < thresh:
        radius += radius_incr
        check = gpd.GeoDataFrame(
            {"check": [0], "geometry": [centroid.buffer(radius)]}, crs="EPSG:4326"
        )
    # print(len(listings.sjoin(check, predicate='intersects'))/len(listings))
    return radius

In [44]:
radii = [
        get_radius(crmls[crmls["lookup_long"] == l], c)
        for l, c in tqdm(
            zip(d["City"], d["centroid"]),
            total=len(d),
        )
    ]
d['radius'] = radii
d.head()

  0%|          | 0/1621 [00:00<?, ?it/s]

,City,latitude,longitude,api_status,centroid,radius,lookup_short
0,Wrightwood,34.360834,-117.633388,OK,POINT (-117.6333884 34.3608341),0.025,WRIW
1,Acton,34.469994,-118.196742,OK,POINT (-118.1967418 34.469994),0.060,AC
2,Agoura,34.153340,-118.761676,OK,POINT (-118.7616764 34.1533395),0.050,AGO
3,Agoura Hills,34.153340,-118.761676,OK,POINT (-118.7616764 34.1533395),0.030,AGHI
4,Agua Dulce,34.496344,-118.326197,OK,POINT (-118.3261969 34.4963443),0.050,ADUL


In [35]:
d = d.merge(lookups, right_on='lookup_long', left_on='City')[list(d.columns) + ['lookup_short']]

In [45]:
d.head()

,City,latitude,longitude,api_status,centroid,radius,lookup_short
0,Wrightwood,34.360834,-117.633388,OK,POINT (-117.6333884 34.3608341),0.025,WRIW
1,Acton,34.469994,-118.196742,OK,POINT (-118.1967418 34.469994),0.060,AC
2,Agoura,34.153340,-118.761676,OK,POINT (-118.7616764 34.1533395),0.050,AGO
3,Agoura Hills,34.153340,-118.761676,OK,POINT (-118.7616764 34.1533395),0.030,AGHI
4,Agua Dulce,34.496344,-118.326197,OK,POINT (-118.3261969 34.4963443),0.050,ADUL


In [53]:
d[d['radius'] != 0]['radius'].describe()

count    1508.000000
mean        0.680975
std         3.497631
min         0.005000
25%         0.025000
50%         0.045000
75%         0.090000
max        54.195000
Name: radius, dtype: float64

In [48]:
d[d['radius'] > 10]

,City,latitude,longitude,api_status,centroid,radius,lookup_short
259,Outside Area (Outside U.S.) Foreign Country,36.778261,-119.417932,OK,POINT (-119.4179324 36.778261),39.750,FC
289,Honolulu,36.778261,-119.417932,OK,POINT (-119.4179324 36.778261),41.440,HNLU
373,Outside Area (Outside Ca),36.778261,-119.417932,OK,POINT (-119.4179324 36.778261),36.615,OS
426,Sequim,36.778261,-119.417932,OK,POINT (-119.4179324 36.778261),11.930,SEQM
487,Avondale,38.520498,-121.411794,OK,POINT (-121.4117936 38.5204984),10.735,AVAZ
515,Carefree,36.778261,-119.417932,OK,POINT (-119.4179324 36.778261),10.925,CFAZ
557,Fair Oaks Ranch,36.778261,-119.417932,OK,POINT (-119.4179324 36.778261),21.965,FOAK
578,Gilbert,38.636940,-120.491670,OK,POINT (-120.49167 38.63694),10.240,GBAZ
708,Prescott,37.807009,-122.297421,OK,POINT (-122.297421 37.8070085),10.350,PRAZ
898,Rosarito,32.366101,-117.061755,OK,POINT (-117.0617553 32.3661011),40.610,ROSAR


In [54]:
d['radius'] = [x if x != 0 else 0.045 for x in d['radius']]

In [56]:
d[['City', 'lookup_short', 'latitude', 'longitude', 'centroid', 'radius']].to_csv('data/city.csv', index=False)

## Getting CountyOrParish Radii

In [121]:
county_centroids = pd.read_csv('data/ca_county_centroids.csv')
county_centroids.head()

,id,name,ancestry_depth,ancestry,centroid_latitude,centroid_longitude
0,37871,Los Angeles County,1,11985,34.196398,-118.261862
1,37874,Mendocino County,1,11985,39.440230,-123.391468
2,37876,Napa County,1,11985,38.506955,-122.330636
3,37877,Plumas County,1,11985,40.004637,-120.838539
4,37878,Sutter County,1,11985,39.034577,-121.694839


In [122]:
crmls.head()

,mls_area_major,city,county_or_parish,latitude,longitude,geometry,city_lookup,county_lookup
0,61,STAN,OR,33.784077,-117.989219,POINT (-117.98922 33.78408),Stanton,Orange
1,82,BP,OR,33.849421,-117.983828,POINT (-117.98383 33.84942),Buena Park,Orange
2,CS,CDM,OR,33.591342,-117.857918,POINT (-117.85792 33.59134),Corona del Mar,Orange
3,TUL,TUL,LA,34.144482,-118.364059,POINT (-118.36406 34.14448),Toluca Lake,Los Angeles
4,TUL,TUL,LA,34.144482,-118.364059,POINT (-118.36406 34.14448),Tulare,Los Angeles


In [123]:
lookups.head()

,resource,field_long,field_short,searchable,data_type,lookup_long,lookup_short,raw_value
0,Property,City,City,True,Character,Wrightwood,WRIW,WRIW
1,Property,City,City,True,Character,Acton,AC,AC
2,Property,City,City,True,Character,Agoura,AGO,AGO
3,Property,City,City,True,Character,Agoura Hills,AGHI,AGHI
4,Property,City,City,True,Character,Agua Dulce,ADUL,ADUL


In [124]:
lookups[lookups['field_long'] == 'CountyOrParish']

,resource,field_long,field_short,searchable,data_type,lookup_long,lookup_short,raw_value
1586,Property,CountyOrParish,CountyOrParish,True,Character,Foreign Country,FC,FC
1587,Property,CountyOrParish,CountyOrParish,True,Character,Alameda,ALA,ALA
1588,Property,CountyOrParish,CountyOrParish,True,Character,Alpine,ALP,ALP
1589,Property,CountyOrParish,CountyOrParish,True,Character,Amador,AMA,AMA
1590,Property,CountyOrParish,CountyOrParish,True,Character,Butte,BUT,BUT
...,...,...,...,...,...,...,...,...
1643,Property,CountyOrParish,CountyOrParish,True,Character,Other State,OS,OS
1644,Property,CountyOrParish,CountyOrParish,True,Character,Other County,OT,OT
1645,Property,CountyOrParish,CountyOrParish,True,Character,Riverside,RI,RI
1646,Property,CountyOrParish,CountyOrParish,True,Character,San Bernardino,SB,SB


In [125]:
crmls.columns = ['mls_area_major', 'city', 'county_or_parish', 'latitude', 'longitude', 'geometry', 'city_lookup', 'lookup_short']
crmls = crmls[['mls_area_major', 'city', 'county_or_parish', 'latitude', 'longitude', 'geometry', 'city_lookup']]

In [126]:
crmls = crmls.merge(lookups[lookups['field_long'] == 'CountyOrParish'], left_on='county_or_parish', right_on='lookup_short')[list(crmls.columns) + ['lookup_long']]

In [127]:
crmls.columns = ['mls_area_major','city','county_or_parish','latitude','longitude','geometry','city_lookup','county_lookup']

In [128]:
crmls.head()

,mls_area_major,city,county_or_parish,latitude,longitude,geometry,city_lookup,county_lookup
0,61,STAN,OR,33.784077,-117.989219,POINT (-117.98922 33.78408),Stanton,Orange
1,82,BP,OR,33.849421,-117.983828,POINT (-117.98383 33.84942),Buena Park,Orange
2,CS,CDM,OR,33.591342,-117.857918,POINT (-117.85792 33.59134),Corona del Mar,Orange
3,TUL,TUL,LA,34.144482,-118.364059,POINT (-118.36406 34.14448),Toluca Lake,Los Angeles
4,TUL,TUL,LA,34.144482,-118.364059,POINT (-118.36406 34.14448),Tulare,Los Angeles


In [129]:
county_centroids.head()

,id,name,ancestry_depth,ancestry,centroid_latitude,centroid_longitude
0,37871,Los Angeles County,1,11985,34.196398,-118.261862
1,37874,Mendocino County,1,11985,39.440230,-123.391468
2,37876,Napa County,1,11985,38.506955,-122.330636
3,37877,Plumas County,1,11985,40.004637,-120.838539
4,37878,Sutter County,1,11985,39.034577,-121.694839


In [130]:
county_centroids['key'] = [x[:-7] for x in county_centroids['name']]
#county_centroids = gpd.GeoDataFrame(county_centroids, crs="EPSG:4326")
county_centroids.head()

,id,name,ancestry_depth,ancestry,centroid_latitude,centroid_longitude,key
0,37871,Los Angeles County,1,11985,34.196398,-118.261862,Los Angeles
1,37874,Mendocino County,1,11985,39.440230,-123.391468,Mendocino
2,37876,Napa County,1,11985,38.506955,-122.330636,Napa
3,37877,Plumas County,1,11985,40.004637,-120.838539,Plumas
4,37878,Sutter County,1,11985,39.034577,-121.694839,Sutter


In [131]:
county_centroids['centroid'] = [
    shapely.Point(long, lat)
    for long, lat in tqdm(zip(county_centroids["centroid_longitude"], county_centroids["centroid_latitude"]), total=len(county_centroids))
]
radii = [
        get_radius(crmls[crmls["county_lookup"] == l], c, radius_incr=0.05)
        for l, c in tqdm(
            zip(county_centroids["key"], county_centroids["centroid"]),
            total=len(county_centroids),
        )
    ]
county_centroids['radius'] = radii

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

In [132]:
county_centroids = lookups[lookups['field_long'] == 'CountyOrParish'].merge(county_centroids, right_on='key', left_on='lookup_long', how='left')[['field_long', 'lookup_long', 'lookup_short', 'name', 'centroid_latitude', 'centroid_longitude', 'radius', 'centroid']]

In [135]:
def get_centroids(listings):
    return shapely.MultiPoint(listings["geometry"].values).centroid
county_centroids['centroid'] = [get_centroids(crmls[crmls["county_lookup"] == l]) if pd.isna(c) else c for l, c in tqdm(
            zip(county_centroids["lookup_long"], county_centroids["centroid"]),
            total=len(county_centroids),
        )]


  0%|          | 0/62 [00:00<?, ?it/s]

In [136]:
county_centroids.head()

,field_long,lookup_long,lookup_short,name,centroid_latitude,centroid_longitude,radius,centroid
0,CountyOrParish,Foreign Country,FC,NaN,NaN,NaN,NaN,POINT (-87.58182937677451 27.582668892297992)
1,CountyOrParish,Alameda,ALA,Alameda County,37.646861,-121.888700,0.45,POINT (-121.8887 37.646861)
2,CountyOrParish,Alpine,ALP,Alpine County,38.597195,-119.820672,0.35,POINT (-119.820672 38.597195)
3,CountyOrParish,Amador,AMA,Amador County,38.446385,-120.651099,0.25,POINT (-120.651099 38.446385)
4,CountyOrParish,Butte,BUT,Butte County,39.666943,-121.600680,0.25,POINT (-121.60068 39.666943)


In [137]:
county_centroids['radius'].describe()

count    58.000000
mean      0.439655
std       0.280626
min       0.100000
25%       0.250000
50%       0.350000
75%       0.525000
max       1.550000
Name: radius, dtype: float64

In [142]:
county_centroids['radius'] = [r if not pd.isna(r) else county_centroids['radius'].mean() for r in county_centroids['radius']]

In [143]:
county_centroids.to_csv('data/county_or_parish.csv')

In [144]:
county_centroids[pd.isna(county_centroids['centroid_latitude'])]

,field_long,lookup_long,lookup_short,name,centroid_latitude,centroid_longitude,radius,centroid
0,CountyOrParish,Foreign Country,FC,NaN,NaN,NaN,0.439655,POINT (-87.58182937677451 27.582668892297992)
49,CountyOrParish,Clark,CLK,NaN,NaN,NaN,0.439655,POINT (-89.68953285714285 38.37555771428571)
57,CountyOrParish,Other State,OS,NaN,NaN,NaN,0.439655,POINT (-96.94079541901282 36.8509664998102)
58,CountyOrParish,Other County,OT,NaN,NaN,NaN,0.439655,POINT (-117.88195547313826 34.72385262977966)
